# **DAILY MACRO RELATIVE VALUE TRADE FRAMEWORK**
### Author:  Deborah Akintoye
#### **Purpose:** Convert one macro headline into a backtestable relative-value trade with clear horizon, factor sensitivities, execution logic, and weekly performance tracking.

---


## 1. Trade Metadata & Idea Structuring

**Date:    21-Jan-2026**

**News Source / Catalyst:** US green energy growth is proving hard to kill

**Markets (s) Impacted:** COM (Energy + Oil Sectors)

**Trade Taxonomy:** RV Sector

**Trade Idea (1 sentence):**
> Long the tech-driven, cost-superior growth of clean energy (ICLN) while shorting the debt-burdened legacy utility model (XLU) to exploit the gap between poltical rhetoric and the actual economic deployment of AI-ready-power.

**Client/Desk Pitch (30 seconds):**
> The market is overreacting to 'anti-green' headlines while ignoring the 2026 reality: solar costs are now 55% lower than natural gas. We are seeing a 46% surge in solar generation because traditional utilities can't build fossil-fuel plants fast enough to cool AI data centeers. By going Long ICLN / Short XLU, we capture the inevitable shift to the cheapest, fastest-to-market energy source while hedging against the broader utility sectors's infrastructure delays and rising fossil-fuel Capex.

**Why Now:**
- AI Power Inflection: Data center demand is not scaling non-linearly, forcing utilities to choose the only energy source that can be deployed in <12 months: Solar
- LCOE Tipping Point: 2026 data confirms solar at $50/MWh vs natural gas at $115/MWh; the economic choice is now decoupled from federal tax credits
- Legal Clarity: Recent court rulings have blocked federal attempts to cancel existing green infrastructure permits, removing the "Policy overhang" risk

---

## 2. Macro Hypothesis & Instruments

### **Macro Hypothesis:**
- Economic Primacy: Profit-seeking utilities are choosing renewables for cost and speed, regardless of the political narrative in Washington
- Battery Scalability: The 2026 battery boom has solved intermittency, allowing solar/wind to function as reliable AI-basedload power
- Fiscal Divergence: Legacy utilities (XLU) are facing balloon debt to maintain aging grids, while clean energy (ICLN) is benefiting from private AI-sector capital

**Long Leg (Tickers + Rationale):**
- ICLN; Captures the 46% growth in solar/wind generation.

**Short Leg (Tickers + Rationale):**
- XLU; Profits from legacy utility stagnation and fossil-fuel Capex drag.

**Weights:** 45% ICLN / 55% XLU (Vol-adjusted to neutralise ICLN's higher beta)

---

## 3. Factor Sensitivity & Horizon Alignment

| Factor                | Strength (S/M/W) | Notes |
|----------------------|------------------|-------|
| USD / FX             |       W           |       |
| Oil Beta             |       M           | Lower oil makes the solar cost advantage narrower      |
| Rates Duration       |       S           | Renewables are highly sensitive to Fed rate paths      |
| Growth vs Value      |       S          | Long Green Growth / Short Legacy Value      |
| Risk-On/Off          |       M           |       |
| China/EM Beta        |       S           | Supply chain for ICLN is China-heavy       |
| Volatility Regime    |       S           | Profits from the transition to a lower-vol solar era      |

**Catalyst Type:** Regime Shift

**Realisation Window:** Weeks / Months (As Q1 '26 utlity earnings confirm data center solar adoption)

**Holding Period Selected:** 120 Days  

**Alignment Score:** Good


---


## 4. Risk Controls & Stop Logic

**Position Sizing:** (units per leg) For a $100,000 Portfolio:
- Long ICLN: 850 Units (~$45k)
- Short XLU: 750 Units (~$55k)

**Stop-Loss:** -4.0% (Triggered if gas prices drop below $1.50/MMBtu or solar tariffs hit 100%) 

**Take-Profit (optional):**  Scale out 50% at +6%; Full exit at +10% 

**Re-entry:** No  

**FX-Hedging Required:** Yes. While both tickers are US-listed, ICLN is inherently multi-currency with approximately 35% expousre to non-USD assets (primarily EUR, DKK, BRL); therefore, if the US Dollar strengthens significantly against these "green" currencies, the Long leg will face a structural drag that the domestic-only Short XLU leg will not offset.

**Liquidity Notes:** We are straading a liquidity mismatch where XLU is ~10x more liquid than ICLN (AUM of $22bn vs $2bn as of Jan 2026); to avoid price slippage you must always execute the ICLN leg first using limit orders and wait for the fill before legging into the highly liquid XLU side

**Failure Mode (tick):**
- [ ] Time
- [ ] Price
- [X] Information

---


# **PYTHON IMPLEMENTATION**

In [20]:
# --- Python Imports ---

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: f'{x:.4f}')


In [21]:
# --- Parameters ---

long_tickers = ["ICLN"]     # EDIT HERE
short_tickers = ["XLU"]                    # EDIT HERE

start_date = "2020-01-01"                         # Backtest window
holding_period = 120                               # days
stop_loss = -0.04                                 # -2% stop
take_profit = 0.06                                # optional e.g. +0.03 for +3%

# --- Weightings ---

weights_long = 0.45
weights_short = 0.55


In [22]:
# --- Data Download ---

tickers = long_tickers + short_tickers
data = yf.download(tickers, start=start_date)["Close"]

# Drop empty cols & make sure no NA

data = data.dropna(how='all').ffill()

returns = data.pct_change().dropna()


/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_33331/911793947.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date)["Close"]
[*********************100%***********************]  2 of 2 completed


In [23]:
# --- Spread Returns ---

long_returns = returns[long_tickers].mean(axis=1)
short_returns = returns[short_tickers].mean(axis=1)

strategy_returns = long_returns - short_returns


In [24]:
# --- Backtest Simulation ---

trade_pnl = []
trade_dates = []

dates = strategy_returns.index
i = 0
n = len(strategy_returns)

while i < n - holding_period:
    pnl = 0.0
    stopped = False
    entry_date = dates[i]
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
        
        if take_profit is not None and pnl >= take_profit:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
    
    if not stopped:
        trade_pnl.append(pnl)
        trade_dates.append(entry_date)
    
    i += holding_period

trade_results = pd.DataFrame({
    "Trade Date": trade_dates,
    "PnL": trade_pnl
})

trade_results.tail()


,Trade Date,PnL
7,2023-05-05,0.0605
8,2023-10-26,-0.0423
9,2024-04-19,-0.0456
10,2024-10-10,-0.0606
11,2025-04-04,0.0853


In [25]:
# --- Performance Summary ---

summary = {
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Best Trade": trade_results["PnL"].max(),
    "Worst Trade": trade_results["PnL"].min(),
    "Stop-Out Frequency": (trade_results["PnL"] <= stop_loss).mean()
}

pd.Series(summary)


Total Trades         12.0000
Win Rate              0.5000
Average PnL           0.0123
Best Trade            0.0853
Worst Trade          -0.0606
Stop-Out Frequency    0.5000
dtype: float64

## Interpretation & Trader Notes

**Absolute Metrics:**
- Total Trades: 12.0000
- Win Rate: 0.5000
- Average PnL: 0.0123
- Best Trade: 0.0853
- Worst Trade: -0.0606
-Stop-Out Frequency: 0.5000

**What went right:**
- The fundamental economics of the theme worked in favourable rate environments → solar/clean energy outperformed legacy utilities.
- Positive right-tail events captured: when 10Y yields softened or paused, ICLN squeezed higher and delivered strong +6% to +8.5% outcomes.
- Sector dispersion was real: Utilities lagged due to balance sheet leverage + capex drag while clean energy rode growth/multiple expansion.
- Asymmetric payoff profile despite coin-flip win rate: Average PnL > 0 with solid right skew → indicates a valid macro RV signal.

**What went wrong:**
- Rates-duration spillover dominated the stop-outs: rising US yields crushed ICLN before the thematic thesis could play out.
- Holding period too long (120D) for a factor-volatile, rates-sensitive expression → forced high stop-out frequency.
- Factor mismatch: Long leg behaves like growth + duration, short leg behaves like value + defensive.
- Execution blind spot: No filter for rate regimes; stops were hit during Fed repricing, not because the thematic thesis failed.

**Factor that actually drove results:**

**US Rates Duration / Real Yields (primary driver)**

- Clean energy = long duration / high sensitivity to discount rates
- Utilities = value + defensive with lower rate sensitivity

**Would I trade this live?** (Yes / No and why)

> Yes. This is a tradeable theme, but the live expression must acknowledge duration risk. The thesis is structurally sound, but the raw ICLN/XLU spread is effectively a Rates + Growth/Value factor trade in the short/medium term.

---

# **BLOOMBERG-STYLE TRADE BLOTTER**

In [26]:
import uuid
from datetime import datetime

def generate_trade_id(prefix="RV"):
    return f"{prefix}-{uuid.uuid4().hex[:8].upper()}"


In [27]:
# --- Computing Synthetic Basket Price Series ---

basket_long = (data[long_tickers] * weights_long).sum(axis=1)
basket_short = (data[short_tickers] * weights_short).sum(axis=1)

basket_spread = basket_long - basket_short


In [28]:
# --- Backtesting Loop Modified for Blotter ---

blotter_rows = []

i = 0
n = len(strategy_returns)
dates = strategy_returns.index

while i < n - holding_period:
    entry_date = dates[i]
    exit_date = dates[min(i + holding_period - 1, n - 1)]
    
    entry_price = basket_spread.loc[entry_date]
    
    pnl = 0.0
    stopped = False
    trade_status = "CLOSED"
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            exit_date = dates[i + j]
            trade_status = "STOPPED"
            stopped = True
            break
    
    if not stopped and take_profit is not None and pnl >= take_profit:
        exit_date = dates[i + j]
        trade_status = "TAKE_PROFIT"
    
    exit_price = basket_spread.loc[exit_date]
    holding_days = (exit_date - entry_date).days
    
    row = {
        "TradeID": generate_trade_id(prefix="RV"),
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Strategy": "Macro RV Spread",
        "Taxonomy": trade_taxonomy if 'trade_taxonomy' in globals() else "RV Macro Spread",
        "LongLeg": ", ".join(long_tickers),
        "ShortLeg": ", ".join(short_tickers),
        "EntryDate": entry_date.strftime("%Y-%m-%d"),
        "ExitDate": exit_date.strftime("%Y-%m-%d"),
        "EntryPrice": entry_price,
        "ExitPrice": exit_price,
        "GrossPnL(%)": pnl * 100,
        "StopLoss(%)": stop_loss * 100,
        "TakeProfit(%)": take_profit * 100 if take_profit else None,
        "Status": trade_status,
        "HoldingPeriod": holding_days,
        "Notes": "",  # Optional notes field
    }
    blotter_rows.append(row)
    
    i += holding_period

blotter = pd.DataFrame(blotter_rows)
blotter.tail()


,TradeID,Timestamp,Strategy,Taxonomy,LongLeg,ShortLeg,EntryDate,ExitDate,EntryPrice,ExitPrice,GrossPnL(%),StopLoss(%),TakeProfit(%),Status,HoldingPeriod,Notes
7,RV-7C49A865,2026-01-21 11:19:35,Macro RV Spread,RV Macro Spread,ICLN,XLU,2023-05-05,2023-08-16,-9.4575,-9.1458,-4.5880,-4.0000,6.0000,STOPPED,103,
8,RV-FCBF0D10,2026-01-21 11:19:35,Macro RV Spread,RV Macro Spread,ICLN,XLU,2023-10-26,2023-11-01,-9.6268,-9.9475,-4.2264,-4.0000,6.0000,STOPPED,6,
9,RV-ACA1CABE,2026-01-21 11:19:35,Macro RV Spread,RV Macro Spread,ICLN,XLU,2024-04-19,2024-05-10,-11.4352,-12.6045,-4.5622,-4.0000,6.0000,STOPPED,21,
10,RV-1F674750,2026-01-21 11:19:35,Macro RV Spread,RV Macro Spread,ICLN,XLU,2024-10-10,2024-10-15,-14.8894,-15.5205,-6.0613,-4.0000,6.0000,STOPPED,5,
11,RV-97DFC869,2026-01-21 11:19:35,Macro RV Spread,RV Macro Spread,ICLN,XLU,2025-04-04,2025-09-25,-15.1681,-16.5465,18.9501,-4.0000,6.0000,TAKE_PROFIT,174,


In [29]:
blotter.to_csv("trade_blotter.csv", index=False)